#Ноутбук для обучения ms-marco моделей на смержинных текстах

In [ ]:
dataset_names = ['initial', '2 same topics', '2 different topics', '3 same topics', '3 different topics']
dataset_paths = ['/content/drive/MyDrive/QA_training/extractive_QA/datasets_csv/ms_initial_train.csv',
                 '/content/drive/MyDrive/QA_training/extractive_QA/merged datasets/ms_marco/merged_by_2_train.csv',
                 '/content/drive/MyDrive/QA_training/extractive_QA/merged datasets/ms_marco/old_merged_datasets/ms_merged_train.csv',
                 '/content/drive/MyDrive/QA_training/extractive_QA/merged datasets/ms_marco/merged_by_3_train.csv',
                 '/content/drive/MyDrive/QA_training/extractive_QA/merged datasets/ms_marco/random_merged_by_3_train.csv']

In [ ]:
model_names = ['3 same topic roberta2roberta', '3 same topic roberta2bert']
model_paths = ['/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/2_same_topics_roberta2roberta',
               '/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/2_same_topics_roberta2roberta_epoch_3_4',
               '/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/2_different_topics_roberta2roberta',
               '/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_same_topics_roberta2roberta',
               '/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_same_topics_roberta2roberta_epoch_3_4',
               '/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_different_topics_roberta2roberta',
               '/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_different_topics_roberta2roberta_epoch_3_4']

# Обучение

In [ ]:
!pip install import-ipynb
%cd '/content/drive/MyDrive/QA_training/generative_QA'
import import_ipynb
from support_functions import *

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=81b0f3255d86ca14d8a0adb353d2fcc155faa543f8c6fd6b4880b6d2903f8947
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
/content/drive/MyDrive/QA_training/generative_QA
importing Jupyter notebook from support_functions.ipynb
     |████████████████████████████████| 2.5MB 4.0MB/s 
     |████████████████████████████████| 3.3MB 34.6MB/s 
     |████████████████████████████████| 901kB 48.7MB/s 
     |████████████████████████████████| 245kB 4.0MB/s 
     |████████████████████████████████| 122kB 17.6MB/s 
     |████████████████████████████████| 245kB 18.3MB/s 


In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

In [ ]:
train_csv_path = dataset_paths[2]
print('Dataset path:', train_csv_path)


train_dataset = prepare_data(train_csv_path, tokenizer,  text_num=1, parts_num = 2, part = 1)

Dataset path: /content/drive/MyDrive/QA_training/extractive_QA/merged datasets/ms_marco/old_merged_datasets/ms_merged_train.csv


In [ ]:
dir = model_paths[2]

#print('Importing model path:', model_v4_paths[number-2])
print('New model path:', dir)

model = EncoderDecoderModel.from_encoder_decoder_pretrained("roberta-base", "roberta-base", 
                                                            bos_token_id=tokenizer.bos_token_id, 
                                                            eos_token_id=tokenizer.eos_token_id, 
                                                            pad_token_id=tokenizer.pad_token_id)


#model = EncoderDecoderModel.from_pretrained(model_paths[0])
#model = EncoderDecoderModel.from_pretrained(dir+'/checkpoint-46000')


New model path: /content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/2_different_topics_roberta2roberta


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.5.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.5.crossattention.output.dense.bias', 'roberta.encoder.layer.6.crossattention.self.va

In [ ]:
training_args = TrainingArguments(
    output_dir=dir,          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    logging_dir=dir+'/logs',            # directory for storing logs
    logging_steps=500,
    save_steps = 2000,
    prediction_loss_only=True,
    overwrite_output_dir = True,
    learning_rate = 1e-5
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    #eval_dataset=val_dataset,            # evaluation dataset

)

In [ ]:
print('Dataset path:', train_csv_path)
print('New model path:', dir)

torch.cuda.empty_cache()
trainer.train()
#trainer.train(dir+'/checkpoint-46000')
trainer.save_model(dir)

Dataset path: /content/drive/MyDrive/QA_training/extractive_QA/merged datasets/ms_marco/old_merged_datasets/ms_merged_train.csv
New model path: /content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/2_different_topics_roberta2roberta


Step,Training Loss
500,7.384700
1000,6.782900
1500,6.557500
2000,6.415700
2500,6.296200
3000,6.191400
3500,6.112300
4000,6.014700
4500,5.904500
5000,5.840100


In [ ]:
  logs = model_paths[0] + '/logs'
logs

'/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/bert2bert/logs'

In [ ]:
!kill 319

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --inspect --logdir bert2bert:'/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/bert2bert/logs'

ERROR: Failed to launch TensorBoard (exited with 0).
Contents of stderr:
2021-05-24 08:11:20.090120: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Contents of stdout:
Processing event files... (this can take a few minutes)

No event files found within logdir bert2bert:/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/bert2bert/logs

In [ ]:
%tensorboard --logdir_spec 3_same_bert2bert:/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_same_topics_bert2bert/logs,3_diff_bert2bert:/content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_defferent_topics_bert2bert/logs

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/QA_training/generative_QA/ms_marco/models/3_same_topics_longformer2roberta_attempt_3/logs